In [35]:
from dotenv import load_dotenv 
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.messages.messages_request_builder import MessagesRequestBuilder
from msgraph.generated.search.query.query_post_request_body import QueryPostRequestBody
from msgraph.generated.search.query.query_request_builder import QueryRequestBuilder
from msgraph.generated.search.search_request_builder import SearchRequestBuilder
from msgraph.generated.models.search_request import SearchRequest
from msgraph.generated.models.entity_type import EntityType
from msgraph.generated.models.search_query import SearchQuery
from msgraph.generated.models.search_response import SearchResponse
import os
import json
import asyncio 
import uuid

In [36]:
from typing import Tuple

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding

In [37]:
kernel = sk.Kernel()
# Configure AI service used by the kernel
api_key, org_id = sk.openai_settings_from_dot_env()
#kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-4", api_key, org_id))
kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x134af0490>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x134af48d0>}

In [38]:
load_dotenv()

True

In [39]:
COLLECTION = "ESPC23"

In [40]:
TENANT_ID = os.getenv("TENANT_ID")
CLIENT_ID = os.getenv("CLIENT_ID")
graphUserScopes = os.getenv("graphUserScopes").split(' ')
gabby = '9c72876c-b0ec-41b3-8079-8ebe216daa34'
print(graphUserScopes)

['User.Read', 'Mail.ReadWrite', 'Chat.ReadWrite', 'Calendars.Read', 'Files.Read.All', 'Sites.Read.All']


In [41]:
credential = DeviceCodeCredential(client_id=CLIENT_ID, tenant_id=TENANT_ID) 
scopes = graphUserScopes
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [44]:
#this will get the client
result = await client.me.get()

In [45]:
print(result.display_name)

Gabby Gabgab


## Do the Query and Get the Intent of the User Asking the Question (UserPrompt)

In [46]:
# note: Using a File to Run these Plugins instead of Inline to 
# show a different way
plugins_directory = "Plugins/"
intentFunctions = kernel.import_semantic_skill_from_directory(plugins_directory, "HoldMyHandPlugin")

getIntentFunction = intentFunctions["GetIntent"]

In [47]:
userIntent = getIntentFunction("What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?")

In [48]:
subject_data = json.loads(str(userIntent))
print(subject_data)

{'subject': 'Gabby', 'action': 'know what was accomplished during the ESPC demo in Antwerp'}


In [49]:
subject = subject_data['subject']
action = subject_data['action']

In [50]:
print(subject)

Gabby


In [51]:
print(action)

know what was accomplished during the ESPC demo in Antwerp


## Work with Teams(Chat) Search Query Endpoint

In [52]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.ChatMessage
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

chat_result = await client.search.query.post(request_body)

In [53]:
chat_data = chat_result.value

In [54]:
##Testing with GUID
search_response = chat_data[0] 

# Initialize an empty list to store summaries, chat IDs, and generated GUIDs
summaries_chat_ids_and_guids = []

# Iterate over hits_containers in the SearchResponse object
for hits_container in search_response.hits_containers:
    # Iterate over each hit in the container
    for hit in hits_container.hits:
        # Extract the summary
        summary = hit.summary

        # Extract the chatId from additional_data
        chat_id = hit.resource.additional_data.get('chatId', 'Unknown')

        # Generate a unique GUID for each hit
        guid = str(uuid.uuid4())

        # Append the chatId, summary, and GUID as a tuple to the list
        summaries_chat_ids_and_guids.append((guid, summary))

# Now 'summaries_chat_ids_and_guids' contains the chatId, summary, and a unique GUID for each search hit
for item in summaries_chat_ids_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=summary)
    print(item)

                                            

('d0093f2e-754b-4a52-ad73-7c7f7930dcf5', 'I prefer dinner in thanksgiving dinner amsterdam this year over thanksgiving dinner in sweden last year...')
('2b279177-0e90-4826-a495-b30d569436bc', "I can't get over the thanksgiving dinner we had in amsterdam.")
('66f751fa-86ee-4d28-b865-284e691750e1', '"id": "9c72876c-b0ec-41b3-8079-8ebe216daa34"')
('5446c9ee-8b06-4357-ba1e-cb4b978da529', 'Fabian Williams  lets practice the demo while we are in Antwerp  sounds like a plan')
('d909f8bf-3507-48b4-909b-0ccf2473af69', "Fabian Williams     hello Gabby, are you prepared for the ChatGPT session with Python?         yes! i'm very excited.")
('90263832-be5c-406d-b5ab-16bc0837bf7d', 'lets practice the demo while we are in Antwerp...')
('52c7b758-868a-4386-9e09-3989c6fe82d6', 'hello Gabby, are you prepared for the ChatGPT session with Python?')


In [20]:
print(summaries_chat_ids_and_guids)

[('2da2acc0-5439-40e6-8f8b-556ecbfe467e', 'I prefer dinner in thanksgiving dinner amsterdam this year over thanksgiving dinner in sweden last year...'), ('c333671d-cfff-44ba-8e92-aa7b71199602', "I can't get over the thanksgiving dinner we had in amsterdam."), ('e4354670-6da5-44b2-8a09-8c6387f4efde', '"id": "9c72876c-b0ec-41b3-8079-8ebe216daa34"'), ('4df53692-b2e7-4076-8eba-6dbe2a1c5ae9', 'Fabian Williams  lets practice the demo while we are in Antwerp  sounds like a plan'), ('2c980560-01f7-41ca-bc9d-582e6b5839b1', "Fabian Williams     hello Gabby, are you prepared for the ChatGPT session with Python?         yes! i'm very excited."), ('6e0b3e03-e932-48e7-9ed2-da895f3b469f', 'lets practice the demo while we are in Antwerp...'), ('a810da58-7be1-4048-be5e-585be616ce39', 'hello Gabby, are you prepared for the ChatGPT session with Python?')]


## Work with Drives(Files) Search Query Endpoint

In [21]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.DriveItem
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

drive_result = await client.search.query.post(request_body)

In [22]:
drive_data = drive_result.value
#print(drive_data)

In [23]:
try:
    drive_search_response = drive_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    drive_summary_from_and_guids = []
    drive_narrative = []

    if drive_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for drive_hits_container in drive_search_response.hits_containers:
            # Iterate over each hit in the container
            for drive_hit in drive_hits_container.hits:
                # Extract the summary
                drive_summary = drive_hit.summary

                # Extract the 'from' information from additional_data
                from_data = drive_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')
                file_name = getattr(drive_hit.resource, 'name', 'Unknown')
                user_email = from_data.get('email', 'Unknown')

                # Generate a unique GUID for each hit
                drive_guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                drive_summary_from_and_guids.append((drive_guid, drive_summary, from_name, from_email))
                drive_narrative.append((drive_guid, file_name))
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'drive_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in drive_narrative:
    await kernel.memory.save_information_async("ESPC23", id=drive_guid, text=file_name)
    print(item)



('b0be1a18-1af4-4e7d-b637-26004d2078d0', 'CollabSummit2024_GabbySpeaking.docx')
('acb2d613-4012-45b7-a8b5-fae4c0771570', 'Apps')
('47b6266a-06d4-4f96-8909-2a006299a9ff', 'ProjectRoundtreeStatusReport.docx')
('13e108aa-470e-4fa0-ac4b-a2deed75ad68', 'HowToGainAccessToStudyGroup.docx')


In [24]:
print(drive_narrative)

[('b0be1a18-1af4-4e7d-b637-26004d2078d0', 'CollabSummit2024_GabbySpeaking.docx'), ('acb2d613-4012-45b7-a8b5-fae4c0771570', 'Apps'), ('47b6266a-06d4-4f96-8909-2a006299a9ff', 'ProjectRoundtreeStatusReport.docx'), ('13e108aa-470e-4fa0-ac4b-a2deed75ad68', 'HowToGainAccessToStudyGroup.docx')]


## Work with Messages(Email) Search Query Endpoint

In [25]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.Message
			],
			query = SearchQuery(
				query_string = subject,
			),
		),
	],
)

mail_result = await client.search.query.post(request_body)

In [26]:
mail_data = mail_result.value

In [27]:
try:
    mail_search_response = mail_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    mail_summary_from_and_guids = []

    if mail_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for mail_hits_container in mail_search_response.hits_containers:
            # Iterate over each hit in the container
            for mail_hit in mail_hits_container.hits:
                # Extract the summary
                mail_summary = mail_hit.summary

                # Extract the 'from' information from additional_data
                from_data = mail_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')

                # Generate a unique GUID for each hit
                guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                mail_summary_from_and_guids.append((guid, mail_summary, from_name, from_email))
                mailconstruct = mail_summary + " from " + from_name
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'mail_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in mail_summary_from_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=mailconstruct)
    print(item)

('a3dba3fd-11d3-49f2-92b6-4c98b2ccd594', "Fabian Williams shared a file with you  Here's the document that Fabian Williams shared with you.   ProjectRoundtreeStatusReport   This link will work for anyone.  Open   Privacy...", 'Unknown', 'Unknown')
('5524525f-89dc-4d32-bf1a-fb62e29203ac', "Fabian Williams shared a file with you   Here's the document that Fabian Williams shared with you.   HowToGainAccessToStudyGroup  1 min to read this.   This link will work for...", 'Unknown', 'Unknown')
('3cf7c94d-87a3-47bf-81f7-769cfa4d79a9', 'From: Microsoft <microsoft-noreply@microsoft.com> Sent: Thursday, November 9, 2023 8:15 PM To: Fabian Williams <fabian@fabster.onmicrosoft.com> Subject: Your Microsoft invoice...', 'Unknown', 'Unknown')
('33824d6a-9041-4cf2-b1d8-8e48a7a8aec6', 'Private to you Hi, <c0>Gabby</c0> Gabgab, Welcome to your new digest from Microsoft Viva Work smarter with Microsoft Viva Insights Explore insights on your work patterns in areas including...', 'Unknown', 'Unknown')
('d8

## Query and Read All 3 Payloads from Semantic Memory ONLY

In [28]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?",
        "What ESPC23 artifacts do I have?",
    ]

    for question in questions:
        results = await kernel.memory.search_async("ESPC23", question, limit=3)
        for result in results:
            print(f"Question: {question}")
            print(f"Answer: {result.text}\nRelevance: {result.relevance}\n")

In [29]:
await search_memory_examples(kernel)

Question: What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?
Answer: hello Gabby, are you prepared for the ChatGPT session with Python?
Relevance: 0.7862505624190529

Question: What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?
Answer: Ok, So make sure you know your lines, study your answers to the questions. Get your demo ready to go. MOST IMPORTANT: do all the sacrifices to the demo gods!!! Dad. from Unknown
Relevance: 0.7582602447035468

Question: What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?
Answer: HowToGainAccessToStudyGroup.docx
Relevance: 0.7546709696624045

Question: What ESPC23 artifacts do I have?
Answer: HowToGainAccessToStudyGroup.docx
Relevance: 0.7298733339489141

Question: What ESPC23 artifacts do I have?
Answer: Ok, So make sure you know your lines, study your answers to the questions. Get your demo ready to go. MOST IMPORTANT: do all the sacrifices to the demo gods!!! Dad

## Setup RAG Function

In [30]:
print(COLLECTION)

ESPC23


In [31]:
async def setup_RAG(kernel: sk.Kernel) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    You are a friendly and very talkative ChatBot. 
    
    {{$history}}
    Answer the {{$user_input}}. You may use information provided in {{$retreived_context}} it may be useful. 
    Use information only from {{$retreived_context}} otherwise say 'Nothing came back from Microsoft Graph'. 
    Do not start off the response with I'm sorry
    ChatBot:""".strip()

    rag_func = kernel.create_semantic_function(sk_prompt, max_tokens=2000, temperature=0.1)

    context = kernel.create_new_context()

    return rag_func, context

In [32]:
async def RAG(kernel: sk.Kernel, rag_func: sk.SKFunctionBase, context: sk.SKContext) -> bool:

    user_input = input("User:> ")
    context["user_input"] = user_input
    #Retrieve
    result = await kernel.memory.search_async(COLLECTION,context["user_input"])

    
    if result[0].text:
        context["retreived_context"] = result[0].text
    
    #Then generate
    answer = await kernel.run_async(rag_func, input_vars=context.variables)

    print(f"\n\u001b[34mChatBot:> {answer}\u001b[0m \n\n\033[1;32m Source: {context['retreived_context']}\n Relevance: {result[0].relevance}\u001b[0m \n")
    return False

In [33]:
print("Simulate Chat experience with SK and RAG")
rag_func, context = await setup_RAG(kernel)

Simulate Chat experience with SK and RAG


In [34]:
print("Ask a question (type 'exit' to exit):\n")
answer = await RAG(kernel, rag_func, context)

Ask a question (type 'exit' to exit):



User:>  What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?



ChatBot:> Based on the information provided, it seems like you and Gabby prepared for a ChatGPT session with Python in Antwerp this weekend. However, I don't have specific details about the ESPC demo. 

 Source: hello Gabby, are you prepared for the ChatGPT session with Python?
 Relevance: 0.7862505624190529 



### User Prompts that work

In [38]:
# GREAT
# What do I need to do to prepare for my ESPC demo
# GOOD
# What did I accomplish this weekend with Gabby in Antwerp regarding our ESPC demo?
# Where did I go this weekend and what did I do
# 
#